In [48]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

mainRowRDD = sc.textFile("./data/small_train.csv") \
               .map(lambda x: x.split(',')) \
               .map(lambda p: Row(msno=p[0], song_id=p[1], source_system_tab=p[2], source_screen_name=p[3], source_type=p[4], target=p[5]))
mainSchema = sqlContext.createDataFrame(mainRowRDD)
mainSchema.registerTempTable("main")
sqlContext.cacheTable("main")
sqlContext.sql('select * from main limit 5').show()
print('\n')

memberRowRDD = sc.textFile("./data/members.csv") \
                 .map(lambda x: x.split(',')) \
                 .map(lambda p: Row(msno=p[0], city=p[1], bd=p[2], gender=p[3], registered_via=p[4], registration_init_time=p[5], expiration_date=p[6]))
memberSchema = sqlContext.createDataFrame(memberRowRDD)
memberSchema.registerTempTable("member")
sqlContext.cacheTable("member")
sqlContext.sql('select * from member limit 5').show()
print('\n')

songRowRDD = sc.textFile("./data/songs.csv") \
               .map(lambda x: x.split(',')) \
               .map(lambda p: Row(song_id=p[0], song_length=p[1], genre_ids=p[2], artist_name=p[3], composer=p[4], lyricist=p[5], language=p[6]))
songSchema = sqlContext.createDataFrame(songRowRDD)
songSchema.registerTempTable("song")
sqlContext.cacheTable("song")
sqlContext.sql('select * from song limit 5').show()

#userRDD = sc.textFile("./data/members.csv").map(lambda x: x.split(',')).cache()
#songRDD = sc.textFile("./data/songs.csv").map(lambda x: x.split(',')).cache()
# print(mainRowRDD.top(5))

msno                 song_id              source_screen_name  source_system_tab source_type     target
FGtllVqz18RPiwJj/... BBzumQNXUHKdEBOB7... Explore             explore           online-playlist 1     
Xumu+NIjS6QYVxDS4... bhp/MpSNoqoxOIB+/... Local playlist more my library        local-playlist  1     
Xumu+NIjS6QYVxDS4... JNWfrrC7zNN7BdMps... Local playlist more my library        local-playlist  1     
Xumu+NIjS6QYVxDS4... 2A87tzfnJTSWqD7gI... Local playlist more my library        local-playlist  1     
FGtllVqz18RPiwJj/... 3qm6XTZ6MOCU11x8F... Explore             explore           online-playlist 1     


bd city expiration_date gender msno                 registered_via registration_init_time
0  1    20170920               XQxgAYj3klVKjR3ox... 7              20110820              
0  1    20170622               UizsfmJb9mV54qE9h... 7              20150628              
0  1    20170712               D8nEhsIOBSoE6VthT... 4              20160411              
0  1    20150907    

In [80]:
sqlContext.sql('select * from main, member where main.msno = member.msno').registerTempTable("temp")
some_df = sqlContext.sql('select * from temp, song where temp.song_id = song.song_id')

In [94]:
#save the table and load
some_df.save("all.parquet", "parquet", mode='overwrite')
all_df = sqlContext.parquetFile("all.parquet")

In [92]:
print(type(all_df.rdd))

<class 'pyspark.rdd.PipelinedRDD'>


In [95]:
from pyspark.mllib.tree import DecisionTree

model = DecisionTree.trainClassifier(all_df.rdd,)

TypeError: trainClassifier() takes at least 4 arguments (2 given)

In [36]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import hashlib

# Loading main table
userIdRDD = mainRDD.map(lambda l: l[0]).distinct().collect()
userIdMap = {}
for s in userIdRDD:
    userIdMap.setdefault(s, len(userIdMap))
    
songIdRDD = mainRDD.map(lambda l: l[1]).distinct().collect()
songIdMap = {}
for s in songRDD:
    songIdMap.setdefault(s, len(songIdMap))

# Loading user table
bigTable = mainRDD.join().userRDD


Mean Squared Error = 0.46965094738


In [ ]:
# Collobartive Filtering
ratings = mainRDD.map(lambda l: Rating(userIdMap[l[0]], songIdMap[l[1]], float(l[5])))

# Build the recommendation model using Alternating Least Squares
rank = 5
numIterations = 5
model = ALS.trainImplicit(ratings, rank, numIterations, alpha=0.01)

# Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

# Save and load model
model.save(sc, "target/tmp/myCollaborativeFilter")
sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")

In [37]:
sameModel.predict(len(userMap)-1, len(songMap)-1)

-0.0003976877393984529